# Lecture 8: Tidy evaluation in R

**Note from Firas**

Our series of R lectures will be presented by Dr. Tiffany Timbers, the other option co-director of the Vancouver MDS program.

Her lecture videos are not yet posted on YouTube, but I've got them locally so that's how we'll watch them.

### What Metaprogramming lets you do in R

- write `library(purrr)` instead of `library("purrr")`
- enable `plot(x, sin(x))` to automatically label the axes with `x` and `sin(x)`
- create a model object via `lm(y ~ x1 + x2, data = df)`
- and much much more (that you will see in Data Wrangling as we explore the tidyverse)

### What is metaprogramming?

Code that writes code/code that mutates code.

### Our narrow focus on metaprogramming for this course:

Tidy evaluation

### Why focus on tidy evaluation

In the rest of MDS you will be relying on functions from the tidyverse to do a lot of:
- data wrangling
- statistics
- data visualization

## The functions from the tidyverse are beautiful to use interactively:

In [1]:
library(gapminder)
library(dplyr)
head(gapminder)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


with base r:

In [2]:
gapminder[gapminder$country == "Canada" & gapminder$year == 1952, ]

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.75,14785584,11367.16


In the tidyverse:

In [ ]:
filter(gapminder, country == "Canada", year == 1952)

### How does that even work?

- When functions like `filter` are called, there is a delay in evaluation and the data frame is temporarily promoted as first class objects, we say the data masks the workspace

- This is to allow the promotion of the data frame, such that it masks the workspace (global environment) 

- When this happens, R can then find the relevant columns for the computation

*This is referred to as data masking*

### We can capture expressions and later evaluate the code:

We will functions from base R and rlang to demonstrate this:

In [6]:
library(rlang)
x <- 10
y <- expr(x)

In [7]:
eval(y)

[1] 10

### We can also manipulate the environment code is evaluated in:

In [9]:
x <- 10
y <- 2
expr(x + y)
eval(expr(x + y), env(x = 1000))

x + y

[1] 1002

### You can evaluate in a data mask:

In [11]:
#head(gapminder)

In [12]:
gapminder$gdpPercap * gapminder$pop

ERROR: Error in eval(expr, envir, enclos): object 'gdpPercap' not found


In [64]:
x <- expr(gdpPercap * pop)
x

gdpPercap * pop

In [66]:
eval(x, gapminder)

[1] 6.567086e+09 7.585449e+09 8.758856e+09 9.648014e+09 9.678553e+09
   [6] 1.169766e+10 1.259856e+10 1.182099e+10 1.059590e+10 1.412200e+10
  [11] 1.836341e+10 3.107929e+10 2.053670e+09 2.867792e+09 3.996989e+09
  [16] 5.476396e+09 7.500110e+09 8.864476e+09 1.009420e+10 1.149842e+10
  [21] 8.307722e+09 1.094591e+10 1.615393e+10 2.137641e+10 2.272563e+10
  [26] 3.095611e+10 2.806140e+10 4.143324e+10 6.173941e+10 8.422742e+10
  [31] 1.150971e+11 1.321197e+11 1.321024e+11 1.394670e+11 1.654477e+11
  [36] 2.074449e+11 1.489956e+10 1.746062e+10 2.060359e+10 2.898060e+10
  [41] 3.226426e+10 1.854132e+10 1.934385e+10 1.913602e+10 2.295683e+10
  [46] 2.248682e+10 3.013483e+10 5.958390e+10 1.056763e+11 1.344666e+11
  [51] 1.518208e+11 1.846882e+11 2.339966e+11 2.719707e+11 2.640107e+11
  [56] 2.890048e+11 3.161041e+11 3.970536e+11 3.372234e+11 5.150336e+11
  [61] 8.725625e+10 1.063492e+11 1.318846e+11 1.724580e+11 2.212238e+11
  [66] 2.580373e+11 2.957428e+11 3.558531e+11 4.095112e+11 5.012233e+11
  [71] 5.998472e+11 7.036584e+11 4.251627e+10 6.159630e+10 7.665118e+10
  [76] 9.468084e+10 1.256987e+11 1.494721e+11 1.635896e+11 1.795277e+11
  [81] 2.140367e+11 2.348005e+11 2.641488e+11 2.962294e+11 1.188461e+09
  [86] 1.613362e+09 2.191816e+09 2.993238e+09 4.216406e+09 5.751940e+09
  [91] 7.261180e+09 8.421244e+09 1.007917e+10 1.214601e+10 1.536203e+10
  [96] 2.111268e+10 3.208206e+10 3.398532e+10 3.901117e+10 4.530627e+10
 [101] 4.459489e+10 5.307281e+10 6.300969e+10 7.802857e+10 9.526285e+10
 [106] 1.199574e+11 1.541591e+11 2.093118e+11 7.283869e+10 8.732886e+10
 [111] 1.013213e+11 1.256588e+11 1.618715e+11 1.877729e+11 2.067837e+11
 [116] 2.223318e+11 2.569225e+11 2.811183e+11 3.143695e+11 3.501412e+11
 [121] 1.847398e+09 1.847398e+09 2.043222e+09 2.514309e+09 2.998327e+09
 [126] 3.260658e+09 4.653596e+09 5.202273e+09 5.934205e+09 7.479327e+09
 [131] 9.645995e+09 1.164315e+10 7.719575e+09 6.833571e+09 7.838236e+09
 [136] 1.045274e+10 1.360781e+10 1.802333e+10 1.780974e+10 1.695274e+10
 [141] 2.041633e+10 2.558864e+10 2.882546e+10 3.485465e+10 2.717131e+09
 [146] 4.164871e+09 5.725731e+09 7.787883e+09 1.092299e+10 1.441737e+10
 [151] 1.721909e+10 1.871884e+10 1.083913e+10 1.719225e+10 2.507154e+10
 [156] 3.389703e+10 3.765108e+08 4.358290e+08 5.043823e+08 6.723914e+08
 [161] 1.401970e+09 2.512321e+09 4.416187e+09 7.144114e+09 1.067930e+10
 [166] 1.328665e+10 1.793969e+10 2.060363e+10 1.193716e+11 1.630498e+11
 [171] 2.537119e+11 3.019989e+11 5.027594e+11 7.613445e+11 9.067173e+11
 [176] 1.115931e+12 1.084077e+12 1.341292e+12 1.462921e+12 1.722599e+12
 [181] 1.778194e+10 2.302010e+10 3.408978e+10 4.634615e+10 5.658143e+10
 [186] 6.696505e+10 7.313032e+10 7.392763e+10 5.457130e+10 4.815750e+10
 [191] 5.897116e+10 7.821393e+10 2.428340e+09 2.909042e+09 3.554493e+09
 [196] 4.075819e+09 4.644538e+09 4.378233e+09 5.355437e+09 6.919414e+09
 [201] 8.272383e+09 9.796843e+09 1.271241e+10 1.743546e+10 8.297895e+08
 [206] 1.012495e+09 1.052082e+09 1.375624e+09 1.638263e+09 2.132331e+09
 [211] 2.563212e+09 3.187458e+09 3.669694e+09 2.835010e+09 3.134234e+09
 [216] 3.608510e+09 1.729534e+09 2.310185e+09 3.023033e+09 3.643124e+09
 [221] 3.141354e+09 3.663575e+09 4.541489e+09 5.725431e+09 6.925441e+09
 [226] 8.652054e+09 1.158525e+10 2.421888e+10 5.873971e+09 7.037837e+09
 [231] 8.108812e+09 9.556814e+09 1.182444e+10 1.419588e+10 2.190581e+10
 [236] 2.805846e+10 2.235567e+10 2.405249e+10 3.080878e+10 3.613752e+10
 [241] 1.680701e+11 2.124560e+11 2.555967e+11 3.347108e+11 4.227497e+11
 [246] 5.256835e+11 5.770931e+11 7.069260e+11 7.513913e+11 8.775034e+11
 [251] 1.063270e+12 1.212704e+12 1.383807e+09 1.657994e+09 1.817614e+09
 [256] 1.969511e+09 2.062194e+09 2.404605e+09 2.369849e+09 2.399456e+09
 [261] 2.442004e+09 2.737291e+09 2.990229e+09 3.084613e+09 3.161727e+09
 [266] 3.787905e+09 4.378505e+09 4.184010e+09 4.304977e+09 4.976221e+09
 [271] 3.889896e+09 5.237128e+09 6.802737e+09 7.599529e+09 1.021572e+10
 [276] 1.744758e+10 2.512768e+10 3.041835e+

The **data mask** allows you to mingle variables from an environment and a data frame in a single expression. 

In [15]:
y <- 1000
new_exp <- expr(pop / y)

eval(new_exp, gapminder)

[1]    8425.333    9240.934   10267.083   11537.966   13079.460   14880.372
   [7]   12881.816   13867.957   16317.921   22227.415   25268.405   31889.923
  [13]    1282.697    1476.505    1728.137    1984.060    2263.554    2509.048
  [19]    2780.097    3075.321    3326.498    3428.038    3508.512    3600.523
  [25]    9279.525   10270.856   11000.948   12760.499   14760.787   17152.804
  [31]   20033.753   23254.956   26298.373   29072.015   31287.142   33333.216
  [37]    4232.095    4561.361    4826.015    5247.469    5894.858    6162.675
  [43]    7016.384    7874.230    8735.988    9875.024   10866.106   12420.476
  [49]   17876.956   19610.538   21283.783   22934.225   24779.799   26983.828
  [55]   29341.374   31620.918   33958.947   36203.463   38331.121   40301.927
  [61]    8691.212    9712.569   10794.968   11872.264   13177.000   14074.100
  [67]   15184.200   16257.249   17481.977   18565.243   19546.792   20434.176
  [73]    6927.772    6965.860    7129.864    7376.998    7544.201    7568.430
  [79]    7574.613    7578.903    7914.969    8069.876    8148.312    8199.783
  [85]     120.447     138.655     171.863     202.182     230.800     297.410
  [91]     377.967     454.612     529.491     598.561     656.397     708.573
  [97]   46886.859   51365.468   56839.289   62821.884   70759.295   80428.306
 [103]   93074.406  103764.241  113704.579  123315.288  135656.790  150448.339
 [109]    8730.405    8989.111    9218.400    9556.500    9709.100    9821.800
 [115]    9856.303    9870.200   10045.622   10199.787   10311.970   10392.226
 [121]    1738.315    1925.173    2151.895    2427.334    2761.407    3168.267
 [127]    3641.603    4243.788    4981.671    6066.080    7026.113    8078.314
 [133]    2883.315    3211.738    3593.918    4040.665    4565.872    5079.716
 [139]    5642.224    6156.369    6893.451    7693.188    8445.134    9119.152
 [145]    2791.000    3076.000    3349.000    3585.000    3819.000    4086.000
 [151]    4172.693    4338.977    4256.013    3607.000    4165.416    4552.198
 [157]     442.308     474.639     512.764     553.541     619.351     781.472
 [163]     970.347    1151.184    1342.614    1536.536    1630.347    1639.131
 [169]   56602.560   65551.171   76039.390   88049.823  100840.058  114313.951
 [175]  128962.939  142938.076  155975.974  168546.719  179914.212  190010.647
 [181]    7274.900    7651.254    8012.946    8310.226    8576.200    8797.022
 [187]    8892.098    8971.958    8658.506    8066.057    7661.799    7322.858
 [193]    4469.979    4713.416    4919.632    5127.935    5433.886    5889.574
 [199]    6634.596    7586.551    8878.303   10352.843   12251.209   14326.203
 [205]    2445.618    2667.518    2961.915    3330.989    3529.983    3834.415
 [211]    4580.410    5126.023    5809.236    6121.610    7021.078    8390.505
 [217]    4693.836    5322.536    6083.619    6960.067    7450.606    6978.607
 [223]    7272.485    8371.791   10150.094   11782.962   12926.707   14131.858
 [229]    5009.067    5359.923    5793.633    6335.506    7021.028    7959.865
 [235]    9250.831   10780.667   12467.171   14195.809   15929.988   17696.293
 [241]   14785.584   17010.154   18985.849   20819.767   22284.500   23796.400
 [247]   25201.900   26549.700   28523.502   30305.843   31902.268   33390.141
 [253]    1291.695    1392.284    1523.478    1733.638    1927.260    2167.533
 [259]    2476.971    2840.009    3265.124    3696.513    4048.013    4369.038
 [265]    2682.462    2894.855    3150.417    3495.967    3899.068    4388.260
 [271]    4875.118    5498.955    6429.417    7562.011    8835.739   10238.807
 [277]    6377.619    7048.426    7961.258    8858.908    9717.524   10599.793
 [283]   11487.112   12463.354   13572.994   14599.929   15497.046   16284.741
 [289]  556263.527  637408.000  665770.000  754550.000  862030.000  943455.000
 [295] 1000281.000 1084035.000 1164970.000 1230075.000 1280400.000 1318683.096
 [301]   12350.771   14485.993   17009.885   19764.02

### Back to our example:

What is going on here?

- code evaluation is delayed 
- the `filter` function quotes columns `country` and `year`
- the `filter` function then creates a data mask (to mingle variables from the environment and the data frame)
- the columns `country` and `year` and unquoted and evaluated within the data mask

In [ ]:
filter(gapminder, country == "Canada", year == 1952)

### Trade off of lovely interactivity of tidyverse functions...

### programming with them can be more challenging.

Let's try writing a function which wraps filter for gapminder:

In [16]:
#filter(gapminder, country == "Canada")

filter_gap <- function(col, val) {
    filter(gapminder, col == val)
}

filter_gap(country, "Canada")

ERROR: Error: object 'country' not found


Why does `filter` work with non-quoted variable names, but our function `filter_gap` fail?

### Proper way of defining this function:

Use `enquo` to quote the column names, and then `!!` to unquote them in context.

In [17]:
filter_gap <- function(col, val) {
    col <- enquo(col)
    filter(gapminder, !!col == val)
}

filter_gap(country, "Canada")

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.750,14785584,11367.16
Canada,Americas,1957,69.960,17010154,12489.95
Canada,Americas,1962,71.300,18985849,13462.49
Canada,Americas,1967,72.130,20819767,16076.59
Canada,Americas,1972,72.880,22284500,18970.57
Canada,Americas,1977,74.210,23796400,22090.88
Canada,Americas,1982,75.760,25201900,22898.79
Canada,Americas,1987,76.860,26549700,26626.52
Canada,Americas,1992,77.950,28523502,26342.88


### Evaluating functions and quoting functions in R

- differ in the way they get their arguments

- evaluating functions take arguments as values

- a quoting function is not passed the value of an expression, it is passed the expression itself

### Evaluating functions 

- take arguments as values:

In [18]:
identity(6)

[1] 6

In [19]:
identity(2 * 3)

[1] 6

In [20]:
a <- 2
b <- 3
identity(a * b)

[1] 6

### Quoting functions 

- take the expression itself, not the value

In [25]:
typeof(quote(6))

[1] "double"

In [26]:
typeof(quote(2 * 3))

[1] "language"

In [27]:
typeof(quote(a * b))
identity(quote(a * b))

[1] "language"

a * b

You get the code! Not the value!

### Not always one or the other:

In practice some functions take both arguments that are evaluated and quoted, for example:

```select(iris, Species)``` 

Here `iris` is an evaluated argument, and `Species` is a quoted argument.

### How can you tell if an argument is quoted?

The argument will not work correctly outside of its original context and ordinary indirect references do not work, some examples:

```library(dplyr)```


In [28]:
temp <- dplyr

ERROR: Error in eval(expr, envir, enclos): object 'dplyr' not found


In [29]:
temp <- "dplyr"
library(temp)

ERROR: Error in library(temp): there is no package called ‘temp’


We get these errors because there is no `dplyr` object for R to find, and `temp` is interpreted by `library` directly as a package name rather than as an indirect reference.

```filter(mtcars, cyl == 4)```

In [30]:
temp <- cyl == 4

ERROR: Error in eval(expr, envir, enclos): object 'cyl' not found


In [31]:
temp <- "cyl" == 4
filter(mtcars, temp)

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


```sum(mtcars2$am)```

In [32]:
temp <- mtcars$am

In [33]:
sum(temp)

[1] 13

It worked! `sum()` is an evaluating function and the indirect reference was resolved in the ordinary way.

R cannot find `cyl` because we haven’t specified where to find it. This object exists only inside the mtcars data frame. And then when we put `temp` in the `subset` function it tries to use `temp` as a column name but it doesn't exist and so we get nothing returned.

### Challenge 1

Which of the function arguments in the function call below are quoted? Which are evaluated?

```arrange(mtcars, cyl)```

### Let's try some quoting and unquoting so we can use indirect references in a dplyr function:

Unquoting is accomplished using the `!!` (pronounced "bang bang") operator.

In [38]:
col1 <- quote(country)
val1 <- "Canada"
col2 <- quote(year)
val2 <- 2007

typeof(col1)
col2

[1] "symbol"

[1] "name"

year

In [35]:
filter(gapminder, !!col1 == val1, !!col2 == val2)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,2007,80.653,33390141,36319.24


### Viewing the unquoted expression:

The `qq_show` function from the `rlang` package performs unquoting and prints the result to the screen:

In [39]:
qq_show(filter(gapminder, !!col1 == val1, !!col2 == val2))

filter(gapminder, country == val1, year == val2)


### Challenge 2

Re-write the code below using quoting and unquoting so that you can create two variables, `var_1` and `var_2`, to indirectly reference the column names in this function call. Also use `rlang::qq_show` to check your expresion.

```arrange(mtcars, hp, mpg)```

In [41]:
var_1 <- quote(hp)
var_2 <- quote(mpg)

qq_show(arrange(mtcars, !!var_1, !!var_2))

arrange(mtcars, hp, mpg)


### `enquo` vs `quote` when writing a function

- `quote` quotes what you typed

- `enquo` quotes what your user typed (i.e., it makes a function argument automatically quote its input)

In [43]:
filter_gap <- function(col, val) {
    col <- enquo(col)
    filter(gapminder, !!col == val)
}

filter_gap(country, "Canada")

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.750,14785584,11367.16
Canada,Americas,1957,69.960,17010154,12489.95
Canada,Americas,1962,71.300,18985849,13462.49
Canada,Americas,1967,72.130,20819767,16076.59
Canada,Americas,1972,72.880,22284500,18970.57
Canada,Americas,1977,74.210,23796400,22090.88
Canada,Americas,1982,75.760,25201900,22898.79
Canada,Americas,1987,76.860,26549700,26626.52
Canada,Americas,1992,77.950,28523502,26342.88


### Quote - unquote pattern all in one step: interpolation

- In the newest release of `rlang`, there has been the introduction of the `{{` (pronounced "curly curly") operator.

- Does the same thing as `enguo` and `!!` but (hopefully) easier to use.

In [44]:
filter_gap <- function(col, val) {
    filter(gapminder, {{col}} == val)
}

filter_gap(country, "Canada")

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.750,14785584,11367.16
Canada,Americas,1957,69.960,17010154,12489.95
Canada,Americas,1962,71.300,18985849,13462.49
Canada,Americas,1967,72.130,20819767,16076.59
Canada,Americas,1972,72.880,22284500,18970.57
Canada,Americas,1977,74.210,23796400,22090.88
Canada,Americas,1982,75.760,25201900,22898.79
Canada,Americas,1987,76.860,26549700,26626.52
Canada,Americas,1992,77.950,28523502,26342.88


### Creating functions that handle unquoted column names:

Or do it all in one step with the brand new curly curly `{{` operator:

In [ ]:
filter_gap <- function(col, val) {
    filter(gapminder, {{col}} == val)
}

filter_gap(country, "Canada")

### Creating functions that handle column names as strings:

Sometimes you want to pass a column name into a function as a string (often useful when you are programming and have the column names as a character vector).

You can do this by using symbols + unquoting with `sym` + `!!` :

In [ ]:
# example of what we want to wrap: filter(gapminder, country == "Canada")
filter_gap <- function(col, val) {
    col <- sym(col)
    filter(gapminder, !!col == val)
}

filter_gap("country", "Canada")

### Another operator is needed when assigning values...

- `:` is needed before the `=` when quoting and unquoting:

In [61]:
library(rlang)

In [71]:
r <- quote(c(1, 2, 3))
eval(r)
{{c(1, 2, 3)}}

?eval_tidy

[1] 1 2 3

[1] 1 2 3

In [63]:
x <- quote(mpg)
typeof(x)
select(mtcars, !!x)

[1] "symbol"

,mpg
,<dbl>
Mazda RX4,21.0
Mazda RX4 Wag,21.0
Datsun 710,22.8
Hornet 4 Drive,21.4
Hornet Sportabout,18.7
Valiant,18.1
Duster 360,14.3
Merc 240D,24.4
Merc 230,22.8


In [48]:
old_col <- quote(mpg)
new_col <- quote(kml)

mutate(mtcars, !!new_col := !!old_col * 0.425144)

#mutate(mtcars, mpg = mpg * 0.425144)

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,kml
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,8.928024
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,8.928024
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,9.693283
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,9.098082
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,7.950193
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,7.695106
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,6.079559
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,10.373514
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,9.693283


In [ ]:
old_col <- quote(mpg)
new_col <- quote(kml)

mutate(mtcars, !!new_col := !!old_col * 0.425144)

### Pass the dots when you can

If you are only passing on variable to a tidyverse function, and that variable is not used in logical comparisons, or in variable assignment, you can get away with passing the dots:

In [49]:
sort_gap <- function(...) {
    arrange(gapminder, ...)
}

sort_gap(year)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Albania,Europe,1952,55.230,1282697,1601.0561
Algeria,Africa,1952,43.077,9279525,2449.0082
Angola,Africa,1952,30.015,4232095,3520.6103
Argentina,Americas,1952,62.485,17876956,5911.3151
Australia,Oceania,1952,69.120,8691212,10039.5956
Austria,Europe,1952,66.800,6927772,6137.0765
Bahrain,Asia,1952,50.939,120447,9867.0848
Bangladesh,Asia,1952,37.484,46886859,684.2442


### Notes on passing the dots

- the dots must be the last function argument
- they are useful because you can add multiple arguments

For example:

In [ ]:
sort_gap <- function(..., x) {
    print(x + 1)
    arrange(gapminder, ...)
}

sort_gap(year, continent, country, 2)

In [ ]:
sort_gap <- function(x, ...) {
    print(x + 1)
    arrange(gapminder, ...)
}

sort_gap(1, year, continent, country)

### Pass the dots is not always the solution...

In [52]:
library(dplyr)
select_n_change <- function(data, ...) {
    out <- select(data, ...)
    mutate(out, ... := ... / mean(...))
}

select_n_change(mtcars, mpg, cyl, disp, hp)

ERROR: Error: object 'mpg' not found


### When passing in different column names to different functions, use quote & unquote:

In [53]:
select_n_change <- function(data, col_range, col_to_change) {
    out <- select(data, {{col_range}})
    mutate(out, {{col_to_change}} := {{col_to_change}} / mean({{col_to_change}}))
}

select_n_change(mtcars, mpg:hp, mpg)

mpg,cyl,disp,hp
<dbl>,<dbl>,<dbl>,<dbl>
1.0452636,6,160.0,110
1.0452636,6,160.0,110
1.1348577,4,108.0,93
1.0651734,6,258.0,110
0.9307824,8,360.0,175
0.9009177,6,225.0,105
0.7117748,8,360.0,245
1.2144968,4,146.7,62
1.1348577,4,140.8,95


## Combine quoting & unquoting with pass the dots:

In [54]:
select_n_change <- function(data, col_to_change, ...) {
    out <- select(data, ...)
    mutate(out, {{col_to_change}} := {{col_to_change}} / mean({{col_to_change}}))
}

select_n_change(mtcars, mpg, mpg, drat, carb)

mpg,drat,carb
<dbl>,<dbl>,<dbl>
1.0452636,3.90,4
1.0452636,3.90,4
1.1348577,3.85,1
1.0651734,3.08,1
0.9307824,3.15,2
0.9009177,2.76,1
0.7117748,3.21,4
1.2144968,3.69,2
1.1348577,3.92,2


In [ ]:
head(mtcars)

### Another operator is needed when assigning values...

In [ ]:
mutate(mtcars, kml = mpg * 0.425144)



### What did we learn?
- quoting and unquoting
- data masking
- without and quoting and unquoting function arguments for quoted functions, R will attempt to evaluate the arguments - potentially out of context.
- introduction to the tidyverse

### Attribution:

- [Tidy evaluation](https://tidyeval.tidyverse.org/) by Lionel Henry & Hadley Wickham
- [Tidy eval in context](https://speakerdeck.com/jennybc/tidy-eval-in-context)  talk by Jenny Bryan
- [Programming in the tidyverse](https://dplyr.tidyverse.org/articles/programming.html) 

In [75]:
iris[c(2, 4)]

Sepal.Width,Petal.Width
<dbl>,<dbl>
3.5,0.2
3.0,0.2
3.2,0.2
3.1,0.2
3.6,0.2
3.9,0.4
3.4,0.3
3.4,0.2
2.9,0.2
